<a href="https://colab.research.google.com/github/kevinOriginal/news-recommender/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data Term Project
- 2020 - 2 CAU Big Data Term Project
- News Recommendation System

import library

In [ ]:
from glob import glob
import math
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt 

## 1. Get Data

Get Crowling Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/newsdata/'
files = sorted(glob('./**/*.parquet' ,recursive=True))

In [ ]:
print(files)

## 2. Data Processing

- parquet 데이터를 pandas로 읽어와 필요한 feature만 가져온다.
- maskedId 의 **** 를 제거하고 idNo와 붙인다.

In [ ]:
data = []
for i in range (0, len(files)):
  df = pd.read_parquet(files[i])
  data_select = df.loc[:, ['idNo','maskedUserId','templateId','objectId','sympathyCount', 'antipathyCount']]

  # maskedId 뒤에 **** 제거
  id_split = data_select['maskedUserId'].str.split("*")
  process_id = id_split.str[0]
  data_select['maskedUserId'] = process_id

  data.append(data_select)
  print(i+1, " data is processing...")

- 각 날짜의 데이터 이어 붙이기

In [ ]:
result = pd.concat([data[0],data[1],data[2],data[3],data[4],data[5],data[6],
                    data[7],data[8],data[9],data[10],data[11],data[12],data[13]], ignore_index=True)

- 결측값 있는 행 제거하기

In [ ]:
# 결측값 데이터 조회 --> idNo에 169621개
result.isnull().sum()

In [ ]:
# 결측값 존재시 drop
notnull_result = result.dropna()  # NULL 값 제거
notnull_result = notnull_result[notnull_result.idNo != ''] # 공백 데이터 제거

In [ ]:
# 결측값이 제거된 것을 확인
notnull_result.isnull().sum()

In [ ]:
# userId 생성 (maskedUserId + idNo)
notnull_result["userId"] = notnull_result['maskedUserId'].map(str) + notnull_result['idNo']
notnull_result = notnull_result.reset_index(drop=True)

In [ ]:
# Sum of 좋아요 + 싫어요, 좋아요/Sum, 싫어요/Sum
notnull_result['like+hate'] = notnull_result['sympathyCount'] + notnull_result['antipathyCount']
notnull_result['likeRatio'] = notnull_result['sympathyCount'] / notnull_result['like+hate']
notnull_result['hateRatio'] = notnull_result['antipathyCount'] / notnull_result['like+hate']

In [ ]:
notnull_result.head() # 마지막 열에 추가된 것을 확인

- 좋아요/싫어요 ratio 가 0인 경우가 대다수 & null값도 많음,,

In [ ]:
notnull_result.isnull().sum() # null값 643918개

In [ ]:
sum(notnull_result['likeRatio'] == 0) # 0값 = 119825

## 3. 댓글 분포 visualization
- 가로 축: user의 댓글 수 (최소 1개 ~ 최대 238개)
- 세로 축: user수 (예: 2주 동안 댓글을 7개 단 user 수) 

In [ ]:
news_df = notnull_result.loc[:, ['userId','objectId']]
news_df = news_df.sort_values(by = ['userId'])
# userid로 묶어서 총 댓글 갯수 확인
news_df = news_df.groupby('userId').count()
news_df = news_df.sort_values(by = ['objectId'])
news_df['userId'] = news_df.index
news_group = news_df.set_index('objectId')
news_group = news_group.groupby('objectId').count()


plt.rcParams['figure.figsize'] = [25, 10]
bar = news_group.plot.bar(grid = False)
plt.xlabel('Reply news count')
plt.ylabel('User count')
plt.show()
plt.savefig('reply count')

##3. Vector 만들기

In [ ]:
# 결측값이 제거된 DataFrame에서 objectId 가져오기
news = notnull_result.loc[:, ['objectId']]
news = news.drop_duplicates(['objectId']) # 중복된 news id 제거
news = news.sort_values(by = ['objectId'])
news = news.reset_index(drop=True)
news.head()

In [ ]:
# 결측값이 제거된 DataFrame에서 userId 가져오기
user = notnull_result.loc[:, ['userId']]
user = user.drop_duplicates(['userId']) # 중복된 user id 제거
user = user.sort_values(by = ['userId'])
user = user.reset_index(drop=True)
user.head()

In [ ]:
# 결측값이 제거된 DataFrame에서 userid와 objectId 가져오기
user_news = notnull_result.loc[:, ['objectId','userId']]
user_news = user_news.sort_values(by = ['userId'])
user_news = user_news.reset_index(drop=True)
user_news.head()

- numpy로 변환

In [ ]:
np_news = news.values
np_user = user.values
np_news_user = user_news.values

- 이건 아닌 것 같다만,.. 일단 코드 지우지 않았슴다..

In [ ]:
news_vector = pd.DataFrame()

for user_idx, user_row in user.iterrows():
  print('>> '+ str(user_idx +1) +'/' + str(len(user)))
  user_row = user_row['userId']
  temp_vector = pd.DataFrame(columns=[user_row])
  for news_idx, news_row in news.iterrows():
    news_row = news_row['objectId']
    # 데이터가 존재하면
    if ((user_news['objectId'] == news_row) & (user_news['userId'] == user_row)).any():
        temp_vector.loc[news_row] = 1
    else:
        temp_vector.loc[news_row] = 0
  
  news_vector = pd.concat([news_vector, temp_vector], axis=1) # temp_vector 붙이기


news_vector.to_csv(path + "/news_vector.csv", index=True, header=True)
news_vector.to_csv(path + "/news_vector2.csv", index=True, header=False)
news_vector.to_csv(path + "/news_vector3.csv", index=False, header=False)
print('csv file upload to google drive!')